# 插件：fy_trans 输运方程求解 

## 主要功能

- 汇总 core_transport.model 和 core_sources.source
- 求解输运方程

In [ ]:
import pathlib
import scipy.constants
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fytok.contexts.tokamak import Tokamak
from fytok.modules.core_profiles import CoreProfiles
from fytok.modules.equilibrium import Equilibrium
from spdm.view import  sp_view
from spdm.core.path import Path
from spdm.core.function import Function
from spdm.core.expression import Variable

## 导入基准数据


In [ ]:
data_path = pathlib.Path("./data/15MA inductive - burn")

astra_profiles = pd.read_excel(
    next(data_path.glob("*ASTRA.xls")).absolute().as_posix(), sheet_name=1, header=10, usecols="B:BN"
)
astra_rho_tor_norm = astra_profiles["x"].values

core_profiles_astra = CoreProfiles(
    f"file+iterprofiles://{next(data_path.glob('*ASTRA.xls')).absolute().as_posix()}#core_profiles"
)

core_profiles_1d_astra = core_profiles_astra.time_slice.current.profiles_1d

equilibrium_astra = Equilibrium(f"file+geqdsk://{next(data_path.glob('**/g*.txt')).absolute().as_posix()}#equilibrium")

## 初始化 Tokamak


In [ ]:
tokamak = Tokamak(
    device="iter",
    shot=900003,
    equilibrium=f"file+geqdsk://{next(data_path.glob('**/g*.txt')).absolute().as_posix()}#equilibrium",
    # core_profiles=f"file+iterprofiles://{next(data_path.glob('*ASTRA.xls')).absolute().as_posix()}#core_profiles",
    core_transport={
        "model": [
            {"code": {"name": "predefined"}},
            {"code": {"name": "fast_alpha"}},
        ]
    },
    core_sources={
        "source": [
            f"file+iterprofiles://{next(data_path.glob('*ASTRA.xls')).absolute().as_posix()}#core_sources/source/0",
            {"code": {"name": "collisional_equipartition"}},
            {"code": {"name": "fusion", "parameters": {"fusion_reactions": ["D(t,n)alpha"], "heating": True}}},
            # {"code": {"name": "radiation"}},
            # {"code": {"name": "synchrotron_radiation"}},
        ]
    },
    transport_solver={
        "code": {
            "name": "fy_trans",
            "parameters": {
                "discontinuity": [0.96, 0.995],
                "dc_pos": 0.96,
                "hyper_diff": 1.0e-5,
                "max_nodes": 512,
                "bc_tol": 1,
                "verbose": 2,
                "units": {
                    "psi_norm": 1,
                    "psi_norm_flux": -0.1,
                    "*/density": 1.0e19,
                    "*/density_flux": 1.0e23,
                    "*/temperature": 1.0e4,
                    "*/temperature_flux": 1.0e26,
                    # "ion/alpha/density": 1.0e14,
                    # "ion/alpha/density_flux": 1.0e16,
                    # "ion/He/density": 1.0e15,
                    # "ion/He/density_flux": 1.0e15,
                },
            },
        },
        "ion_thermal": ["D", "T"],  #
        "ion_non_thermal": ["alpha", "He"],
        "impurities": ["Ar", "Be"],
        "boundary_condition_type": {
            # fmt: off
            # psi                          at axis \frac{d\psi}{dx}=0  , at boundary \psi=?
            "psi"                       : 1, 
            "psi_norm"                  : 1, 
            # density                      at axis \Gamma_s=0          , at boundary n=?
            "*/density"                 : 1, 
            # temperatur                   at axis \frac{dT}{dx}=0     , at boundary T=?
            "*/temperature"             : 1,
            # fmt: on
        },
    },
)

In [ ]:
tokamak.initialize()

In [ ]:
tokamak.equilibrium.refresh()
tokamak.core_profiles.refresh()
tokamak.core_sources.refresh()
tokamak.core_transport.refresh()

## 检查系数


### 源项 predefined


In [ ]:
source_1d = tokamak.core_sources.source[0].fetch(core_profiles_1d_astra).profiles_1d
fig = sp_view.plot(
    source_1d.grid.rho_tor_norm,
    *[(ion.particles, rf"$S_{{{ion.label}}}$") for ion in source_1d.ion],
    *[(source_1d.electrons.energy, r"$Q_{e}$")] + [(ion.energy, rf"$Q_{{{ion.label}}}$") for ion in source_1d.ion],
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=10,
)

### 源项 碰撞热交换


In [ ]:
source_1d = tokamak.core_sources.source[1].fetch(core_profiles_1d_astra).profiles_1d
fig = sp_view.plot(
    source_1d.grid.rho_tor_norm,
    [(ion.particles, rf"$S_{{{ion.label}}}$") for ion in source_1d.ion],
    [(source_1d.electrons.energy, r"$Q_{e}$")] + [(ion.energy, rf"$Q_{{{ion.label}}}$") for ion in source_1d.ion],
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=8,
)

In [ ]:
fig = sp_view.plot(
    source_1d.grid.rho_tor_norm,
    [
        (-source_1d.electrons.energy, {"label": r"$-Q_{e}$", "$matplotlib": {"marker": "+"}}),
        (sum([ion.energy for ion in source_1d.ion], 0), r"$Q_{ion}$"),
    ],
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

In [ ]:
fig.get_size_inches()

In [ ]:
plt.plot(
    astra_rho_tor_norm,
    -astra_profiles["Peic"].values * 1.0e6 / scipy.constants.electron_volt,
    "--",
    label=r"$-Q_{eic}$ astra",
    
)
plt.plot(source_1d.grid.rho_tor_norm, source_1d.electrons.energy(source_1d.grid.rho_tor_norm), label=r"$Q_{eic}$ fytok")
plt.legend()

In [ ]:
fig=sp_view.plot(
    source_1d.grid.rho_tor_norm,
    [
        (
            (astra_rho_tor_norm, astra_profiles["Peic"].values * 1.0e6 / scipy.constants.electron_volt),
            {"label": r"$Q_{eic}$ astra", "$matplotlib": {"linestyle": "--"}},
        ),
        (source_1d.ion["D"].energy * 2, r"$Q_{ie}$ fytok"),
    ],
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

### 源项 Fusion


In [ ]:
source_1d = tokamak.core_sources.source[2].fetch(core_profiles_1d_astra).profiles_1d
fig = sp_view.plot(
    source_1d.grid.rho_tor_norm,
    [(ion.particles, rf"$S_{{{ion.label}}}$") for ion in source_1d.ion],
    [(source_1d.electrons.energy, r"$Q_{e}$")] + [(ion.energy, rf"$Q_{{{ion.label}}}$") for ion in source_1d.ion],
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=8,
)

In [ ]:
plt.plot(
    astra_rho_tor_norm,
    astra_profiles["Pdt"].values * 1.0e6 / scipy.constants.electron_volt,
    "--",
    label=r"$Q_{DT}$ astra",
)


plt.plot(
    source_1d.grid.rho_tor_norm,
    source_1d.electrons.energy(source_1d.grid.rho_tor_norm)
    + source_1d.ion["D"].energy(source_1d.grid.rho_tor_norm)
    + source_1d.ion["T"].energy(source_1d.grid.rho_tor_norm),
    "x",
    label="$Q$ fusion stix",
)

plt.plot(
    source_1d.grid.rho_tor_norm,
    (
        source_1d.ion["alpha"].particles(source_1d.grid.rho_tor_norm)
        + source_1d.ion["He"].particles(source_1d.grid.rho_tor_norm)
    )
    * (3.54e6),
    label="$S_{He}* 3.54Mev$ fusion",
)

plt.legend()

In [ ]:
plt.plot(
    astra_rho_tor_norm,
    astra_profiles["Pdte"].values * 1.0e6 / scipy.constants.electron_volt,
    "--",
    label=r"$Q_e$ astra",
)
plt.plot(
    source_1d.grid.rho_tor_norm,
    source_1d.electrons.energy(source_1d.grid.rho_tor_norm),
    label="$Q_e$ fusion stix ",
)

plt.plot(
    astra_rho_tor_norm,
    astra_profiles["Pdti"].values / 2 * 1.0e6 / scipy.constants.electron_volt,
    "-.",
    label=r"$Q_D$ astra",
)
plt.plot(
    source_1d.grid.rho_tor_norm,
    source_1d.ion["D"].energy(source_1d.grid.rho_tor_norm),
    label="$Q_D$ fusion stix ",
)
plt.plot(
    source_1d.grid.rho_tor_norm,
    source_1d.ion["T"].energy(source_1d.grid.rho_tor_norm),
    label="$Q_T$ fusion stix ",
)
plt.legend()

### 输运系数 Predefined


In [ ]:
trans_1d = tokamak.core_transport.model[0].fetch(core_profiles_1d_astra).profiles_1d
trans_1d.grid_d = core_profiles_1d_astra.grid
fig = sp_view.plot(
    trans_1d.grid_d.rho_tor_norm,
    ([(ion.particles.d, ion.label) for ion in trans_1d.ion], {"y_label": "D"}),
    ([(ion.particles.v, ion.label) for ion in trans_1d.ion], {"y_label": "v"}),
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

In [ ]:
trans_1d = tokamak.core_transport.model[1].fetch(core_profiles_1d_astra).profiles_1d
trans_1d.grid_d = core_profiles_1d_astra.grid
ion = trans_1d.ion["alpha"]


In [ ]:
fig = sp_view.plot(
    trans_1d.grid_d.rho_tor_norm,
    [
        (ion.particles.d, "D"),
        (-ion.particles.v, r"|v|"),
    ],
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
    height=5,
)

In [ ]:
trans_1d = tokamak.core_transport.model[1].fetch(core_profiles_1d_astra).profiles_1d
trans_1d.grid_d = core_profiles_1d_astra.grid
fig = sp_view.plot(
    trans_1d.grid_d.rho_tor_norm,
    ([(ion.particles.d, ion.label) for ion in trans_1d.ion], {"y_label": "D"}),
    ([(ion.particles.v, ion.label) for ion in trans_1d.ion], {"y_label": "v"}),
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

## 求解


In [ ]:
solver = tokamak.transport_solver
rho_tor_norm_bdry=0.995
rho_tor_norm = np.linspace(0.001,rho_tor_norm_bdry, 128)
eq_grid = tokamak.equilibrium.time_slice.current.profiles_1d.grid.remesh(rho_tor_norm)
psi_norm = eq_grid.psi_norm

In [ ]:
F = (1 - rho_tor_norm**2) ** 2
n =3e19 -tokamak.core_sources.source[0].fetch(core_profiles_1d_astra).profiles_1d.ion["D"].particles.I(rho_tor_norm)


In [ ]:
solver_1d = solver.preprocess(
    time=0.0,
    rho_tor_norm=rho_tor_norm,
    impurity_fraction=0.02 * 4 + 0.0012 * 18,
    # fmt:off
    initial_value={
        "psi":                              0.0,
        "psi_norm":                         psi_norm,
        "ion/D/density":                    n,          # 5.0e19*Fn , #core_profiles_1d_astra.ion["D"].density(rho_tor_norm) ,# 1.0e19,  #
        "ion/T/density":                    n,          # 5.0e19*Fn , #core_profiles_1d_astra.ion["T"].density(rho_tor_norm) ,# 1.0e19,  #
        "ion/He/density":                   2.0e19*F,   # core_profiles_1d_astra.ion["He"].density(rho_tor_norm)   ,#    
        "ion/alpha/density":                1.0e18*F,   #core_profiles_1d_astra.ion["alpha"].density(rho_tor_norm) ,# 
        
        "ion/D/temperature":                3.5e4*F, # core_profiles_1d_astra.ion["D"].temperature(rho_tor_norm) *0.5,# 
        "ion/T/temperature":                3.5e4*F, # core_profiles_1d_astra.ion["T"].temperature(rho_tor_norm) *0.5,# 
        "electrons/temperature":            3.5e4*F, #core_profiles_1d_astra.electrons.temperature(rho_tor_norm),# 
    },
    # boundary_value={
    #     "ion/D/density":                    n[-1],   # core_profiles_1d_astra.ion["D"].density (rho_tor_norm[-1])    ,
    #     "ion/T/density":                    n[-1],   # core_profiles_1d_astra.ion["T"].density (rho_tor_norm[-1])    ,
    #     "ion/He/density":                   1.0e19*F[-1],
    #     "ion/alpha/density":                1.0e18*F[-1],
    #     "ion/D/temperature":                4.0e4*F[-1] , # core_profiles_1d_astra.ion["D"].temperature (rho_tor_norm[-1]) ,  # 2000,
    #     "ion/T/temperature":                4.0e4*F[-1] , # core_profiles_1d_astra.ion["T"].temperature (rho_tor_norm[-1]) ,  # 2000,
    #     "electrons/temperature":            4.0e4*F[-1] , # core_profiles_1d_astra.electrons.temperature(rho_tor_norm[-1]),   # 2000,
    # },
    # fmt:on
)

### 检查初值

In [ ]:
fig = sp_view.plot(
    solver_1d.X,
    [
        (solver_1d.Y[0] * solver._units[0], r"$\bar{\psi}$"),
        (
            Function(
                equilibrium_astra.time_slice.current.profiles_1d.grid.rho_tor_norm,
                equilibrium_astra.time_slice.current.profiles_1d.grid.psi_norm,
            ),
            {"label": r"$\bar{\psi}$ astra", "$matplotlib": {"linestyle": "dashed"}},
        ),
    ],
    *[
        [
            (solver_1d.Y[idx * 2] * solver._units[idx * 2], equ.identifier),
            (
                Path(equ.identifier).get(core_profiles_1d_astra),
                {"label": f"{equ.identifier} astra", "$matplotlib": {"linestyle": "dashed"}},
            ),
        ]
        for idx, equ in enumerate(solver.equations)
        if equ.identifier != "psi_norm"
    ],
    vline={"x": 0.96, "color": "r", "linestyle": "--"},
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=8,
)

### 计算

In [ ]:
solver_1d = solver.execute(solver_1d)

In [ ]:
solver_1d = solver.postprocess(solver_1d)

### 检查结果


In [ ]:
fig = sp_view.plot(
    solver_1d.X,
    [
        (solver_1d.Y[0] * solver._units[0], r"$\bar{\psi}$"),
        (
            Function(
                equilibrium_astra.time_slice.current.profiles_1d.grid.rho_tor_norm,
                equilibrium_astra.time_slice.current.profiles_1d.grid.psi_norm,
            ),
            {"label": r"$\bar{\psi}$ astra", "$matplotlib": {"linestyle": "dashed"}},
        ),
    ],
    *[
        [
            (solver_1d.Y[idx * 2] * solver._units[idx * 2], equ.identifier),
            (
                Path(equ.identifier).get(core_profiles_1d_astra),
                {"label": f"{equ.identifier} astra", "$matplotlib": {"linestyle": "dashed"}},
            ),
        ]
        for idx, equ in enumerate(solver.equations)
        if equ.identifier != "psi_norm"
    ],
    (
        (0.5 * (solver_1d.X[:-1] + solver_1d.X[1:]), solver_1d.rms_residuals * 100),
        {"label": "rms residuals", "y_label": r"[\%]"},
    ),
    vline={"x": 0.96, "color": "r", "linestyle": "--"},
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

In [ ]:
fig = sp_view.plot(
    solver_1d.X,
    *[
        [
            (solver_1d.Y[idx * 2] * solver._units[idx * 2], f"{equ.identifier} fytok"),
            (
                Path(equ.identifier).get(core_profiles_1d_astra, 0),
                {"label": f"{equ.identifier} astra", "$matplotlib": {"linestyle": "dashed"}},
            ),
        ]
        for idx, equ in enumerate(solver.equations)
        if equ.identifier.endswith("density")
    ],
    vline={"x": 0.96, "color": "r", "linestyle": "--"},
    fontsize=8,
    x_label=r"$\bar{\rho}$ [-]",
)

In [ ]:
fig = sp_view.plot(
    solver_1d.X,
    *[(solver_1d.Y[idx * 2], equ.identifier) for idx, equ in enumerate(solver.equations)],
    *[(solver_1d.Y[idx * 2 + 1], f"{equ.identifier}_flux") for idx, equ in enumerate(solver.equations)],
    vline={"x": 0.96, "color": "r", "linestyle": "--"},
    x_label=r"$\bar{\rho}$ [-]",
)

In [ ]:
fig = sp_view.plot(
    solver_1d.X,
    *[
        (solver_1d.Yp[idx * 2], rf"$\frac{{d }}{{d\bar{{\rho}}}}$  {equ.identifier} ")
        for idx, equ in enumerate(solver.equations)
    ],
    *[
        (
            solver_1d.Yp[idx * 2 + 1],
            rf"$\frac{{d}}{{d\bar{{\rho}}}}$ {equ.identifier}_flux ",
        )
        for idx, equ in enumerate(solver.equations)
    ],
    vline={"x": 0.96, "color": "r", "linestyle": "--"},    x_label=r"$\bar{\rho}$ [-]",

)

## 后处理


In [ ]:
tokamak.flush()
core_profiles_1d = tokamak.core_profiles.time_slice.current.profiles_1d

In [ ]:
fig = sp_view.plot(
    solver_1d.grid.rho_tor_norm,
    [
        (core_profiles_1d.psi_norm, r"$\bar{\psi}$"),
        (
            Function(
                equilibrium_astra.time_slice.current.profiles_1d.grid.rho_tor_norm[:-1],
                equilibrium_astra.time_slice.current.profiles_1d.grid.psi_norm[:-1],
            ),
            {"label": r"$\bar{\psi}$ astra", "$matplotlib": {"linestyle": "dashed"}},
        ),
    ],
    *[
        [
            (Path(equ.identifier).get(core_profiles_1d, 0), f"{equ.identifier} "),
            (
                Path(equ.identifier).get(core_profiles_1d_astra, 0),
                {"label": f"{equ.identifier} astra", "$matplotlib": {"linestyle": "dashed"}},
            ),
        ]
        for equ in solver_1d.equations
        if equ.identifier != "psi_norm"
    ],
    (
        (0.5 * (solver_1d.X[:-1] + solver_1d.X[1:]), solver_1d.rms_residuals * 100),
        {"label": "$rms$ residuals", "y_label": r"[\%]"},
    ),
    vline={"x": 0.96, "color": "r", "linestyle": "--"},
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

In [ ]:
fig = sp_view.plot(
    solver_1d.grid.rho_tor_norm,
    [
        (Path(equ.identifier).get(core_profiles_1d, 0), f"{equ.identifier} ")
        for equ in solver_1d.equations
        if equ.identifier.endswith("density")
    ],
    vline={"x": 0.96, "color": "r", "linestyle": "--"},
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

In [ ]:
fig = sp_view.plot(
    solver_1d.grid.rho_tor_norm,
    [
        (Path(equ.identifier).get(core_profiles_1d, 0), f"{equ.identifier} ")
        for equ in solver_1d.equations
        if equ.identifier.endswith("density")
    ],
    vline={"x": 0.96, "color": "r", "linestyle": "--"},
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

In [ ]:
fig = sp_view.plot(
    solver_1d.grid.rho_tor_norm,
    [
        (Path(equ.identifier).get(core_profiles_1d, 0), f"{equ.identifier} ")
        for equ in solver_1d.equations
        if equ.identifier.endswith("temperature")
    ],
    vline={"x": 0.96, "color": "r", "linestyle": "--"},
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

In [ ]:
fig = sp_view.plot(
    solver_1d.grid.rho_tor_norm,
    [
        # (core_profiles_1d.ion["alpha"].density, r"$\alpha$"),
        (core_profiles_1d.ion["D"].density, r"n_{D}"),
        (core_profiles_1d.ion["T"].density, r"n_{T}"),
        (core_profiles_1d.ion["He"].density, r"n_{He}"),
        (core_profiles_1d.ion["He"].density + core_profiles_1d.ion["alpha"].density, r"He+$\alpha$"),
        (core_profiles_1d_astra.ion["He"].density, {"label": f"He astra", "$matplotlib": {"linestyle": "dashed"}}),
    ],
    fontsize=10,
    x_label=r"$\bar{\rho}$ [-]",
)

In [ ]:
source_1d = tokamak.core_sources.source[0].time_slice.current.profiles_1d
fig = sp_view.plot(
    source_1d.grid.rho_tor_norm,
    [(source_1d.electrons.particles, r"$S_{e}$")] + [(ion.particles, rf"$S_{{{ion.label}}}$") for ion in source_1d.ion],
    [(source_1d.electrons.energy, r"$Q_{e}$")],
    [(ion.energy, rf"$Q_{{{ion.label}}}$") for ion in source_1d.ion],
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

In [ ]:
source_1d = tokamak.core_sources.source[1].time_slice.current.profiles_1d
fig = sp_view.plot(
    source_1d.grid.rho_tor_norm,
    # [
    #     # (core_profiles_1d.electrons.temperature, r"$T_{e}$"),
    #     *[(ion.temperature, rf"$T_{{{ion.label}}}$") for ion in core_profiles_1d.ion],
    # ],  # [(source_1d.electrons.particles, r"$S_{e}$")] + [(ion.particles, rf"$S_{{{ion.label}}}$") for ion in source_1d.ion],
    [(source_1d.electrons.energy, r"$Q_{e}$")] + [(ion.energy, rf"$Q_{{{ion.label}}}$") for ion in source_1d.ion],
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

In [ ]:
source_1d = tokamak.core_sources.source[2].time_slice.current.profiles_1d

fig = sp_view.plot(
    source_1d.grid.rho_tor_norm,
    # astra_rho_tor_norm,
    [
        (source_1d.electrons.energy, r"$Q_{e}$"),
        # *[(ion.energy, rf"$Q_{{{ion.label}}}$") for ion in source_1d.ion],
        # (
        #     Function(astra_rho_tor_norm, astra_profiles["Pdte"].values * 1.0e6 / scipy.constants.electron_volt),
        #     {"label": r"$Q_{dte}$", "$matplotlib": {"linestyle": "-", "marker": "."}},
        # ),
        # (
        #     Function(astra_rho_tor_norm, astra_profiles["Pdti"].values * 1.0e6 / scipy.constants.electron_volt),
        #     r"$Q_{dti}$ astra",
        # ),
        # (
        #     Function(astra_rho_tor_norm, (astra_profiles["Pdt"].values) * 1.0e6 / scipy.constants.electron_volt),
        #     r"$Q_{dt}$ astra total",
        # ),
    ]
    + [(ion.energy, rf"$Q_{{{ion.label}}}$") for ion in source_1d.ion if ion.label in ["D", "T"]],
    # (
    #     Function(astra_rho_tor_norm, astra_profiles["Pdti"].values * 1.0e6 / scipy.constants.electron_volt/2),
    #     {"label": r"$Q_{dti}$", "$matplotlib": {"linestyle": "-", "marker": "."}},
    # ),
    # (
    #     Function(astra_rho_tor_norm, (astra_profiles["Pdt"].values) * 1.0e6 / scipy.constants.electron_volt),
    #     r"$Q_{dt}$ astra total",
    # ),
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=12,
)

In [ ]:
source_1d = tokamak.core_sources.source[0].time_slice.current.profiles_1d

fig = sp_view.plot(
    # source_1d.grid.rho_tor_norm,
    astra_rho_tor_norm,
    (
        Function(astra_rho_tor_norm, astra_profiles["Poh"].values * 1.0e6 / scipy.constants.electron_volt),
        r"$Q_{oh}$ astra",
    ),
    # [
    #     (tokamak.core_sources.source[1].time_slice.current.profiles_1d.electrons.energy, r"$Q_{e}$"),
    #     *[
    #         (ion.energy, rf"$Q_{{{ion.label}}}$")
    #         for ion in tokamak.core_sources.source[1].time_slice.current.profiles_1d.ion
    #     ],
    # ],
    [
        (
            Function(astra_rho_tor_norm, astra_profiles["Pdte"].values * 1.0e6 / scipy.constants.electron_volt),
            r"$Q_{dte}$ astra",
        ),
        (
            Function(astra_rho_tor_norm, astra_profiles["Pdti"].values * 1.0e6 / scipy.constants.electron_volt),
            r"$Q_{dti}$ astra",
        ),
        (
            Function(astra_rho_tor_norm, (astra_profiles["Pdt"].values) * 1.0e6 / scipy.constants.electron_volt),
            r"$Q_{dt}$ astra total",
        ),
    ],
    [
        (
            Function(
                astra_rho_tor_norm,
                (
                    astra_profiles["Poh"].values
                    + astra_profiles["Paux"].values
                    # - astra_profiles["Prad"].values
                    - astra_profiles["Pneu"].values
                    - astra_profiles["Peic"].values
                    + astra_profiles["Pdte"].values
                )
                * 1.0e6
                / scipy.constants.electron_volt,
            ),
            r"$Q_{e}$ astra",
        ),
        (source_1d.electrons.energy, r"$Q_{e}$"),
    ],
    x_label=r"$\bar{\rho}$ [-]",
    fontsize=10,
)

In [ ]:
plt.plot(source_1d.grid.rho_tor_norm, source_1d.electrons.energy.__array__(), label="Qe")
plt.plot(source_1d.electrons.energy.domain.dims[0], source_1d.electrons.energy.__array__())
plt.plot(
    astra_rho_tor_norm,
    (
        astra_profiles["Poh"].values
        + astra_profiles["Paux"].values
        # - astra_profiles["Prad"].values
        - astra_profiles["Pneu"].values
        # - astra_profiles["Peic"].values
        # + astra_profiles["Pdte"].values
    )
    * 1.0e6
    / scipy.constants.electron_volt,
    label="Qe astra",
)
plt.legend()

In [ ]:
fig = sp_view.plot(
    solver_1d.grid.rho_tor_norm,
    *sum(
        [
            [
                # (solver_1d.Y0[2 * idx], f"d({equ.identifier})/dr"),
                # (solver_1d.Y0[2 * idx + 1], f"d({equ.identifier}_flux)/dr"),
                (equ.d_dr, f"d({equ.identifier})/dr"),
                (equ.dflux_dr, f"d({equ.identifier}_flux)/dr"),
            ]
            for idx, equ in enumerate(solver_1d.equations)
        ],
        [],
    ),
    x_label=r"$\bar{\rho}$ [-]",
)